In [1]:
import gym
env = gym.make("CarRacing-v2", continuous=False)
env.reset()

<frozen importlib._bootstrap>:228: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.
/home/doolyboi/miniconda3/envs/tf/lib/python3.9/site-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/home/doolyboi/miniconda3/envs/tf/lib/python3.9/site-packages/pkg_resources/__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_nam

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [2]:
import random

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

/home/doolyboi/miniconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:246: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),
/home/doolyboi/miniconda3/envs/tf/lib/python3.9/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:326: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),


In [3]:
states = env.observation_space.shape
actions = env.action_space.n

print(states)
print(actions)

(96, 96, 3)
5


In [4]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [5]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [7]:
model = build_model(states[0], states[1], states[2], actions)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 3, 23, 23, 32)     6176      
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 10, 10, 64)     32832     
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 8, 8, 64)       36928     
                                                                 
 flatten (Flatten)           (None, 12288)             0         
                                                                 
 dense (Dense)               (None, 512)               6291968   
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 5)                 1

In [8]:
import numpy as np
import os
import pickle

agent = build_agent(model, actions)
agent.compile(Adam(lr=0.001), metrics=['mae'])

/home/doolyboi/miniconda3/envs/tf/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [9]:
'''
from tensorflow.keras.models import load_model
load_folder_path = '/mnt/c/users/haval/OneDrive - Oregon Institute of Technology/Documents/CST407/project/car_racing'
model = load_model(os.path.join(load_folder_path, 'model.h5'))
agent.load_weights(os.path.join(load_folder_path, 'weights.h5f'))
with open(os.path.join(load_folder_path, 'agent_state.pkl'), 'rb') as f:
  agent_state = pickle.load(f)
agent.memory = agent_state['memory']
agent.model = agent_state['model']
agent._DQNAgent__policy = agent_state['policy']
'''

"\nfrom tensorflow.keras.models import load_model\nload_folder_path = '/mnt/c/users/haval/OneDrive - Oregon Institute of Technology/Documents/CST407/project/car_racing'\nmodel = load_model(os.path.join(load_folder_path, 'model.h5'))\nagent.load_weights(os.path.join(load_folder_path, 'weights.h5f'))\nwith open(os.path.join(load_folder_path, 'agent_state.pkl'), 'rb') as f:\n  agent_state = pickle.load(f)\nagent.memory = agent_state['memory']\nagent.model = agent_state['model']\nagent._DQNAgent__policy = agent_state['policy']\n"

In [10]:
iterations = 20
steps_per_iteration = 100000
base_dir = '/mnt/c/users/haval/OneDrive - Oregon Institute of Technology/Documents/CST407/project/car_racing'


for i in range(1, iterations + 1):
  agent.fit(env, nb_steps=steps_per_iteration, visualize=False, verbose=1)
  
  folder_name = f'car_racing_{(i * steps_per_iteration)}'
  folder_path = os.path.join(base_dir, folder_name)
  os.makedirs(folder_path, exist_ok=True)

  save_path_weights = os.path.join(folder_path, 'weights.h5f')
  agent.save_weights(save_path_weights)

  save_path_model = os.path.join(folder_path, 'model.h5')
  model.save(save_path_model)

  save_path_agent = os.path.join(folder_path, 'agent_state.pkl')
  agent_state = {
    'memory': agent.memory,
    'model': agent.model,
    'policy': agent._DQNAgent__policy
  }
  with open(save_path_agent, 'wb') as f:
    pickle.dump(agent_state, f)

  print(f'Saved stuff to {save_path}')

Training for 100000 steps ...
Interval 1 (0 steps performed)


/home/doolyboi/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   11/10000 [..............................] - ETA: 2:45 - reward: 0.4772

/home/doolyboi/miniconda3/envs/tf/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


 2868/10000 [=======>......................] - ETA: 17:59 - reward: -0.0524done, took 434.482 seconds
[TIP] Next time specify overwrite=True!


...

INFO:tensorflow:Assets written to: ram://7923be4e-d917-4493-bdde-1ace2dfeb033/assets


INFO:tensorflow:Assets written to: ram://1d3189e9-f41e-44ad-97fc-a82f07121617/assets


INFO:tensorflow:Assets written to: ram://1d3189e9-f41e-44ad-97fc-a82f07121617/assets


INFO:tensorflow:Assets written to: ram://dacc924c-869a-44ec-b23c-aad5cb9a011d/assets


INFO:tensorflow:Assets written to: ram://dacc924c-869a-44ec-b23c-aad5cb9a011d/assets


In [ ]:
scores = agent.test(env, nb_episodes=5, visualize=True)
print(np.mean(scores.history["episode_reward"]))

Testing for 5 episodes ...


/home/doolyboi/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/home/doolyboi/miniconda3/envs/tf/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/home/doolyboi/miniconda3/envs/tf/lib/python3.9/site-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


Episode 1: reward: -92.958, steps: 1000
Episode 2: reward: -93.548, steps: 1000


KeyboardInterrupt: 

In [52]:
import gym 
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
import os

In [53]:
a2c_path = '/mnt/c/users/haval/OneDrive - Oregon Institute of Technology/Documents/CST407/project/car_racing/car_racing_100000.zip'

In [54]:
env = make_vec_env('CarRacing-v2', n_envs=1, seed=0)
env = VecFrameStack(env, n_stack=5)

In [55]:
model = A2C("CnnPolicy", env, verbose=1)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [56]:
del model

In [57]:
model = A2C.load(a2c_path, env)

Wrapping the env in a VecTransposeImage.


In [58]:
evaluate_policy(model=model, env=env, n_eval_episodes=10, render=True)

KeyboardInterrupt: 

In [59]:

total_rewards = []
for episode in range(10):
    obs = env.reset()
    done = False
    episode_reward = 0
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(action)
        episode_reward += reward
        env.render()
    total_rewards.append(episode_reward)
    print(f"Episode {episode + 1}: Reward = {episode_reward}")


Episode 1: Reward = [-92.881714]
Episode 2: Reward = [-93.33248]
Episode 3: Reward = [-92.3946]


KeyboardInterrupt: 